In [ ]:
# Download CIFAR-10
import torchvision
torchvision.datasets.CIFAR10(root='./data', train=True, download=True)
torchvision.datasets.CIFAR10(root='./data', train=False, download=True)

In [2]:
# Create SageMaker Session
import sagemaker

use_local_mode = False
default_bucket = 'aws-terraform-a-sagemaker-test'

if use_local_mode:
    raise NotImplementedError("Currently local mode code return CalledProcessError. I will solve this later.")
    from sagemaker.local import LocalSession

    class MyLocalSession(LocalSession):
        def __init__(self, default_bucket):
            super().__init__()
            self._default_bucket=default_bucket

    sagemaker_session = MyLocalSession(default_bucket=default_bucket)
    instance_type = "local"
else:
    from sagemaker import Session

    sagemaker_session = sagemaker.Session(default_bucket=default_bucket)
    instance_type = "ml.g4dn.4xlarge"

In [ ]:
# Upload the data to S3 bucket

bucket = sagemaker_session.default_bucket()
prefix = 'data/cifar10'

role = sagemaker.get_execution_role()
print(role)

inputs = sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)
print(inputs)

In [ ]:
# Show entrypoint script
!pygmentize src/cifar10.py

In [6]:
# Create Estimator
from sagemaker.pytorch import PyTorch

cifar10_estimator = PyTorch(py_version='py3',
                            entry_point='src/cifar10.py',
                            role=role,
                            framework_version='1.7.1',
                            instance_count=1,
                            sagemaker_session=sagemaker_session,
                            instance_type=instance_type,
                            )

In [ ]:
# Kick traing job
cifar10_estimator.fit(inputs)